# Working with MySQL using Python
First we need to install a driver to allow Python to talk with MySQL. Since there are many drivers (https://wiki.python.org/moin/MySQL) we will choose one that has been around a long time and is considered stable -- and works with Python 3. Here are the installation instructions.

sudo apt-get install mysql-server

conda install pymysql

Example 3 does everything - creates a database, creates tables and populates them. Examples 1 and 2 assumes that you have created the database and tables beforehand. Example 4 shoes how to leverage the fetchall command.

In [1]:
# Example 1
# How to access an existing table in a database
# This assumes that you have the Student database in the classwork database
import pymysql as myDB

# Establish a connection to the database
db = myDB.connect(host='localhost', user='root', passwd='root', db='classwork')

# Creeate a cursor. In this case we name it cur
# A cursor is a data structure that receives results for further processing
cur = db.cursor()

# Execute an SQL statement
cur.execute('SELECT * FROM Student')

# the command fetchall returns the data from the last query or command
for row in cur.fetchall():
    print (row)

('S1001', 'Smith', 'Tom', 'History', 90)
('S1002', 'Chin', 'Ann', 'Math', 36)
('S1005', 'Lee', 'Perry', 'History', 3)
('S1010', 'Burns', 'Edward', 'Art', 63)
('S1013', 'McCarthy', 'Owen', ' Math', 0)
('S1015', 'Jones', 'Mary', 'Math', 42)
('S1020', 'Rivera', 'Jane', 'CSC', 15)


In [2]:
#Example 2
# Getting data from an SQL database into a pandas dataframe
import pymysql as myDB
import pandas.io.sql as pdSQL
# import pandas as pd
conn = myDB.connect('localhost', 'root', 'root', 'classwork') 

myDataFrame = pdSQL.read_sql('SELECT * FROM Student', conn) 
print (myDataFrame)
# Now you have access to all the functionality of a pandas dataframe

   stuId  lastName firstName    major  credits
0  S1001     Smith       Tom  History       90
1  S1002      Chin       Ann     Math       36
2  S1005       Lee     Perry  History        3
3  S1010     Burns    Edward      Art       63
4  S1013  McCarthy      Owen     Math        0
5  S1015     Jones      Mary     Math       42
6  S1020    Rivera      Jane      CSC       15


In [ ]:
# Example 3
# This will recreate the example that we did in class
# That is, it will create four tables and populate them
# The variable sql will be used to store the SQL commands that we want
# to execute
# The function cursor.execute() will execute the desired command
import pymysql as myDB

conn = myDB.connect('localhost', 'root', 'root') 
cursor = conn.cursor()

sql = ' SHOW DATABASES; '
cursor.execute(sql)

sql = ' DROP DATABASE IF EXISTS classwork; ' 
cursor.execute(sql)

sql = ' CREATE DATABASE classwork; ' 
cursor.execute(sql)

sql = ' USE classwork; ' 
cursor.execute(sql)

cursor.close()
cursor = conn.cursor()

sql = '''
        CREATE TABLE Student ( stuId CHAR(6),
        lastName CHAR(20) NOT NULL,
        firstName CHAR(20) NOT NULL, major CHAR(10),
        credits SMALLINT DEFAULT 0,
        CONSTRAINT Student_stuId_pk PRIMARY KEY (stuId),
        CONSTRAINT
        Student_credits_cc CHECK ((CREDITS>=0) AND (credits < 150))
        );
    '''
cursor.execute(sql)

sql = '''
        INSERT INTO Student VALUES ("S1001", "Smith", "Tom", "History", 90); 
        INSERT INTO Student VALUES ("S1002", "Chin", "Ann", "Math", 36);
        INSERT INTO Student VALUES ("S1005", "Lee", "Perry", "History", 3); 
        INSERT INTO Student VALUES ("S1010", "Burns", "Edward", "Art", 63); 
        INSERT INTO Student VALUES ("S1013", "McCarthy", "Owen", "Math", 0); 
        INSERT INTO Student VALUES ("S1015", "Jones", "Mary", "Math", 42); 
        INSERT INTO Student VALUES ("S1020", "Rivera", "Jane", "CSC", 15);
    '''
cursor.execute(sql)

cursor.close()


cursor = conn.cursor()

sql = '''
        CREATE TABLE Faculty ( facId CHAR(6),
        name CHAR(20) NOT NULL,
        department CHAR(20) NOT NULL, rank CHAR(10),
        CONSTRAINT Faculty_facId_pk PRIMARY KEY (facId)
        );
    '''
cursor.execute(sql)

sql = '''
        INSERT INTO Faculty VALUES ("F101", "Adams", "Art", "Professor"); 
        INSERT INTO Faculty VALUES ("F105", "Tanaka", "CSC", "Instructor"); 
        INSERT INTO Faculty VALUES ("F110", "Byrne", "Math", "Assistant"); 
        INSERT INTO Faculty VALUES ("F115", "Smith", "History", "Associate"); 
        INSERT INTO Faculty VALUES ("F221", "Smith", "CSC", "Professor");
    '''
cursor.execute(sql)

cursor.close()

cursor = conn.cursor()

sql = '''
        CREATE TABLE Class (
        classNumber CHAR(8), facId CHAR(6) NOT NULL,
        schedule CHAR(8), room CHAR(6),
        CONSTRAINT Class_classNumber_pk PRIMARY KEY (classNumber),
        CONSTRAINT Class_facId_fk FOREIGN KEY (facId)
        REFERENCES Faculty (facId) ON DELETE NO ACTION
        );
    '''
cursor.execute(sql)

sql = '''
        INSERT INTO Class VALUES ("ART103A", "F101", "MWF9", "H221"); 
        INSERT INTO Class VALUES ("CSC201A", "F105", "TuThF10", "M110"); 
        INSERT INTO Class VALUES ("CSC203A", "F105", "MThF12", "M110"); 
        INSERT INTO Class VALUES ("HST205A" ,"F115", "MWF11", "H221"); 
        INSERT INTO Class VALUES ("MTH101B", "F110", "MTuTh9", "H225"); 
        INSERT INTO Class VALUES ("MTH103C", "F110", "MWF11", "H225");
    '''
cursor.execute(sql)

cursor.close()
cursor = conn.cursor()

sql = '''
        CREATE TABLE Enroll (
        classNumber CHAR(8), stuId CHAR(6),
        grade CHAR(2),
        CONSTRAINT Enroll_classNumber_stuId_pk PRIMARY KEY (classNumber, stuId),
        CONSTRAINT Enroll_classNumber_fk FOREIGN KEY (classNumber)
        REFERENCES Class (classNumber) ON DELETE NO ACTION, CONSTRAINT Enroll_stuId_fk
        FOREIGN KEY (stuId)
        REFERENCES Student (stuId) ON DELETE CASCADE
        );
    '''
cursor.execute(sql)

sql = '''
        INSERT INTO Enroll VALUES ("ART103A", "S1001", "A"); 
        INSERT INTO Enroll VALUES ("HST205A", "S1001", "C"); 
        INSERT INTO Enroll VALUES ("ART103A", "S1002", "D"); 
        INSERT INTO Enroll VALUES ("CSC201A", "S1002", "F"); 
        INSERT INTO Enroll VALUES ("MTH103C", "S1002", "B"); 
        INSERT INTO Enroll VALUES ("ART103A", "S1010", NULL); 
        INSERT INTO Enroll VALUES ("MTH103C", "S1010", NULL); 
        INSERT INTO Enroll VALUES ("CSC201A", "S1020", "B"); 
        INSERT INTO Enroll VALUES ("MTH101B", "S1020", "A");
    '''
cursor.execute(sql)
cursor.close()

In [ ]:
# Example 4
# How to see results of SQL commands
import pymysql as myDB
conn = myDB.connect('localhost', 'root', 'root', 'classwork') 
cursor = conn.cursor()

sql = ' SHOW DATABASES; '

cursor.execute(sql) # but the data that you want is not returned

# Option 1: is to return data from last query
databases = cursor.fetchall()
print (databases) # Now print the databases

# Option 2: Iterate over the existing cursor
for (database_name,) in cursor:
    print (database_name)

In example 3 we have used the following pair of commands repeatedly

In [ ]:
cursor.close()
cursor = conn.cursor()

This is because mysql has a problem of processing multiple SQL queries together, It is not a Python problem. If we fail to close and reestablish the connection (which I believe simulates flushing the connection buffer), we get the following error:

In [ ]:
Error 2014: Commands out of sync; you can't run this command now

NOTE: It is useful to know where the database files are stored in Ubuntu. The databases are stored in
/var/lib/mysql

Just as an aside, if you want to read a csv file into mysql, you can refer to  http://stackoverflow.com/questions/10154633/load-csv-data-into-mysql-in-python or the somewhat easier approach shown in https://news.ycombinator.com/item?id=8236644,